# 0. Instalacion de utilitarios Pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.3.4/spark-2.3.4-bin-hadoop2.7.tgz
!tar xf spark-2.3.4-bin-hadoop2.7.tgz

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


In [ ]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 53kB/s 
     |████████████████████████████████| 204kB 50.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=b956117dc975da02796cce1a1cbae38f545e6e53714f0f1c81b2c5b5fc7fda0f
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

# 1. Lectura del origen datos

In [ ]:
# Conexion al drive
#---------------------------
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
ls

drive/        spark-2.3.4-bin-hadoop2.7/     spark-2.4.4-bin-hadoop2.7/
sample_data/  spark-2.3.4-bin-hadoop2.7.tgz  spark-2.4.4-bin-hadoop2.7.tgz


#2.  Cargando los datos en formato Spark

In [ ]:
DS_peliculas_rating = spark.read.csv(
    '/content/drive/My Drive/PySpark Machine Learning en plataforma Big Data/Training R ML en Casos de Negocios/8_Recomendacion_comercio/Comercio_tipo_rappi.csv', 
    sep=';', 
    header=True, 
    inferSchema=True
)

# 2.1 Revision de formatos 
DS_peliculas_rating.printSchema()

root
 |-- userID: string (nullable = true)
 |-- userID_cod: integer (nullable = true)
 |-- placeID: integer (nullable = true)
 |-- Nombre_comerce: string (nullable = true)
 |-- Distrito: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- food_rating: integer (nullable = true)
 |-- service_rating: integer (nullable = true)
 |-- rating_total: integer (nullable = true)



In [ ]:
# Dimensiona de base
print((DS_peliculas_rating.count(), len(DS_peliculas_rating.columns)))

(50766, 9)


In [ ]:
# 2.2 Revisiones  
DS_peliculas_rating.show(5)

+------+----------+-------+--------------------+---------------+------+-----------+--------------+------------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating|food_rating|service_rating|rating_total|
+------+----------+-------+--------------------+---------------+------+-----------+--------------+------------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|     2|          2|             2|           6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|     2|          2|             1|           5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|     2|          2|             2|           6|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|     1|          2|             2|           5|
| U1068|         2| 135104|                vips|              0|     1|          1|             2|           4|
+------+----------+-------+--------------------+---------------+------+-----------+--------------+------

In [ ]:
# 2.3 Validacion de carga del total de registros
print(DS_peliculas_rating.count())

50766


In [ ]:
# 2.4 Exploracion inicial
DS_peliculas_rating.describe().show()

+-------+------+------------------+------------------+--------------------+---------+------------------+------------------+------------------+-----------------+
|summary|userID|        userID_cod|           placeID|      Nombre_comerce| Distrito|            rating|       food_rating|    service_rating|     rating_total|
+-------+------+------------------+------------------+--------------------+---------+------------------+------------------+------------------+-----------------+
|  count| 50766|             50766|             50766|               50766|    50766|             50766|             50766|             50766|            50766|
|   mean|  null| 67.34026710790687|  134254.559390143|                null|      0.0|1.2218216916834101| 1.244277666154513| 1.109856990899421|3.575956348737344|
| stddev|  null|32.640689951647815|1079.9216692471784|                null|      0.0|0.7706200957794772|0.7863420037824467|0.7915679876580894|2.083855254035862|
|    min| U1001|                 1

In [ ]:
# 2.5 Los clientes que más vistaron la plataforma de videos
DS_peliculas_rating.groupBy('userId').count().orderBy('count', ascending=False).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|U1106 |1198 |
|U1071 |949  |
|U1024 |933  |
|U1016 |897  |
|U1036 |876  |
|U1104 |868  |
|U1009 |759  |
|U1037 |755  |
|U1084 |730  |
|U1132 |730  |
+------+-----+
only showing top 10 rows



In [ ]:
# 2.6  Las clientes que menos vieron las peliculas
DS_peliculas_rating.groupBy('userId').count().orderBy('count', ascending=True).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|U1011 |57   |
|U1039 |57   |
|U1129 |76   |
|U1130 |76   |
|U1065 |95   |
|U1043 |95   |
|U1138 |114  |
|U1102 |114  |
|U1077 |117  |
|U1131 |120  |
+------+-----+
only showing top 10 rows



In [ ]:
# 2.7  Las peliculas con mayor numero de visualizaciones
DS_peliculas_rating.groupBy('Nombre_comerce').count().orderBy('count',ascending=False).show(10,False)

+----------------------------------+-----+
|Nombre_comerce                    |count|
+----------------------------------+-----+
|Tortas Locas Hipocampo            |1641 |
|Cafeteria y Restaurant El Pacifico|1466 |
|puesto de tacos                   |1387 |
|La Cantina Restaurante            |1202 |
|Restaurant la Chalita             |1200 |
|Gorditas Doa Gloria               |1176 |
|Restaurante Marisco Sam           |1017 |
|Restaurant Oriental Express       |955  |
|Restaurante El Cielo Potosino     |945  |
|Luna Cafe                         |836  |
+----------------------------------+-----+
only showing top 10 rows



In [ ]:
# Peliculas con menos numero de visualizaciones 
DS_peliculas_rating.groupBy('Nombre_comerce').count().orderBy('count', ascending=True).show(10,False)

+-------------------------------------+-----+
|Nombre_comerce                       |count|
+-------------------------------------+-----+
|emilianos                            |99   |
|Carnitas Mata  Calle 16 de Septiembre|100  |
|cafe ambar                           |100  |
|la perica hamburguesa                |102  |
|tortas hawai                         |109  |
|puesto de gorditas                   |109  |
|tacos los volcanes                   |118  |
|churchs                              |126  |
|TACOS EL GUERO                       |133  |
|Arrachela Grill                      |135  |
+-------------------------------------+-----+
only showing top 10 rows



In [ ]:
# El cliente 100 y el 450 cold celd
DS_peliculas_rating.groupBy(
    'Distrito'
).count().orderBy(
    'count', ascending=False
).show(10, False)

+---------------+-----+
|Distrito       |count|
+---------------+-----+
|San Luis Potosi|33108|
|0              |4283 |
|Cuernavaca     |3824 |
|san luis potosi|1969 |
|victoria       |1642 |
|s.l.p.         |1387 |
|Jiutepec       |1001 |
|Soledad        |705  |
|Ciudad Victoria|629  |
|s.l.p          |497  |
+---------------+-----+
only showing top 10 rows



In [ ]:
DS_peliculas_rating.filter(
    (DS_peliculas_rating['Nombre_comerce'] == 'Luna Cafe')
).groupBy(
    'userId'
).count().orderBy(
    'count', ascending=False
).show(10, False)

+------+-----+
|userId|count|
+------+-----+
|U1049 |73   |
|U1005 |73   |
|U1024 |73   |
|U1104 |69   |
|U1045 |69   |
|U1054 |61   |
|U1075 |49   |
|U1126 |41   |
|U1106 |41   |
|U1086 |40   |
+------+-----+
only showing top 10 rows



In [ ]:

DS_peliculas_rating.groupBy(
    'rating_total'
).count().orderBy(
    'count', ascending=False
).show(10, False)

+------------+-----+
|rating_total|count|
+------------+-----+
|6           |13423|
|3           |9707 |
|0           |8115 |
|4           |7757 |
|5           |6324 |
|2           |3790 |
|1           |1650 |
+------------+-----+



# 3. FEature Engineering: Tansforma pelicula a numero



In [ ]:
# Importando el codifocador
from pyspark.ml.feature import StringIndexer, IndexToString

In [ ]:
# Configurando la Numerzación los titulos de las peliculas
# title_new = La codificación de las peliculas

stringIndexer = StringIndexer(inputCol="Nombre_comerce", outputCol="Nombre_comerce_ID")

In [ ]:
DS_peliculas_rating.show(10)

+------+----------+-------+--------------------+---------------+------+-----------+--------------+------------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating|food_rating|service_rating|rating_total|
+------+----------+-------+--------------------+---------------+------+-----------+--------------+------------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|     2|          2|             2|           6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|     2|          2|             1|           5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|     2|          2|             2|           6|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|     1|          2|             2|           5|
| U1068|         2| 135104|                vips|              0|     1|          1|             2|           4|
| U1068|         2| 132740|Carreton de Flaut...|Ciudad Victoria|     0|          0|             0|      

In [ ]:
DS_peliculas_rating = DS_peliculas_rating.drop('rating', 'food_rating', 'service_rating')
DS_peliculas_rating = DS_peliculas_rating.withColumn('rating', DS_peliculas_rating.rating_total)
DS_peliculas_rating.show()

+------+----------+-------+--------------------+---------------+------------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|
+------+----------+-------+--------------------+---------------+------------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|           6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|           5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|           6|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|           5|
| U1068|         2| 135104|                vips|              0|           4|
| U1068|         2| 132740|Carreton de Flaut...|Ciudad Victoria|           0|
| U1068|         2| 132663|           tacos abi|       victoria|           3|
| U1068|         2| 132732|  Taqueria EL amigo |    Cd Victoria|           0|
| U1068|         2| 132630|          palomo tec|       victoria|           3|
| U1067|         3| 132584|  Gorditas Dona Tota|              0|

In [ ]:
DS_peliculas_rating = DS_peliculas_rating.withColumn('rating', DS_peliculas_rating.rating_total)
DS_peliculas_rating.show()

+------+----------+-------+--------------------+---------------+------------+------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|
+------+----------+-------+--------------------+---------------+------------+------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|           6|     6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|           5|     5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|           6|     6|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|           5|     5|
| U1068|         2| 135104|                vips|              0|           4|     4|
| U1068|         2| 132740|Carreton de Flaut...|Ciudad Victoria|           0|     0|
| U1068|         2| 132663|           tacos abi|       victoria|           3|     3|
| U1068|         2| 132732|  Taqueria EL amigo |    Cd Victoria|           0|     0|
| U1068|         2| 132630|          palomo tec|       victoria| 

In [ ]:
#Aplicando la numerización de peliculas
model = stringIndexer.fit(DS_peliculas_rating)

In [ ]:
#Creando un nuevo Dataframe con el indexado creado
indexed = model.transform(DS_peliculas_rating)

In [ ]:
#Revision de los indices
indexed.show(10)

+------+----------+-------+--------------------+---------------+------------+------+-----------------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|Nombre_comerce_ID|
+------+----------+-------+--------------------+---------------+------------+------+-----------------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|           6|     6|              0.0|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|           5|     5|              4.0|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|           6|     6|              2.0|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|           5|     5|              6.0|
| U1068|         2| 135104|                vips|              0|           4|     4|             80.0|
| U1068|         2| 132740|Carreton de Flaut...|Ciudad Victoria|           0|     0|             72.0|
| U1068|         2| 132663|           tacos abi|       victoria|         

In [ ]:
#Numero de veces que cada titulo ha aparecido
indexed.groupBy('Nombre_comerce_ID').count().orderBy('count',ascending=False).show(10, False)

+-----------------+-----+
|Nombre_comerce_ID|count|
+-----------------+-----+
|0.0              |1641 |
|1.0              |1466 |
|2.0              |1387 |
|3.0              |1202 |
|4.0              |1200 |
|5.0              |1176 |
|6.0              |1017 |
|7.0              |955  |
|8.0              |945  |
|9.0              |836  |
+-----------------+-----+
only showing top 10 rows



# 4. Dividiendo la data en muestra de entrenamiento y validacion


In [ ]:
help(indexed.randomSplit)

Help on method randomSplit in module pyspark.sql.dataframe:

randomSplit(weights, seed=None) method of pyspark.sql.dataframe.DataFrame instance
    Randomly splits this :class:`DataFrame` with the provided weights.
    
    :param weights: list of doubles as weights with which to split the DataFrame. Weights will
        be normalized if they don't sum up to 1.0.
    :param seed: The seed for sampling.
    
    >>> splits = df4.randomSplit([1.0, 2.0], 24)
    >>> splits[0].count()
    1
    
    >>> splits[1].count()
    3
    
    .. versionadded:: 1.4



In [ ]:
DS_peliculas_rating.show(3)

+------+----------+-------+--------------------+---------------+------------+------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|
+------+----------+-------+--------------------+---------------+------------+------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|           6|     6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|           5|     5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|           6|     6|
+------+----------+-------+--------------------+---------------+------------+------+
only showing top 3 rows



In [ ]:
#4.1  Data de entrenamiento y validacion
train, test = indexed.randomSplit([0.75, 0.25], seed=1234)

In [ ]:
# Nro de entrenamiento
train.count()

38060

In [ ]:
train.show(5)

+------+----------+-------+---------------+--------+------------+------+-----------------+
|userID|userID_cod|placeID| Nombre_comerce|Distrito|rating_total|rating|Nombre_comerce_ID|
+------+----------+-------+---------------+--------+------------+------+-----------------+
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
+------+----------+-------+---------------+--------+------------+------+-----------------+
only showing top 5 rows



In [ ]:
# Nro de Validacion
test.count()

12706

In [ ]:
test.show(5)

+------+----------+-------+---------------+--------+------------+------+-----------------+
|userID|userID_cod|placeID| Nombre_comerce|Distrito|rating_total|rating|Nombre_comerce_ID|
+------+----------+-------+---------------+--------+------------+------+-----------------+
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
+------+----------+-------+---------------+--------+------------+------+-----------------+
only showing top 5 rows



# 5. Entrenando el Modelo


In [ ]:
#Importando el modelo Recomendador ALS
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS


In [ ]:
help(ALS)

In [ ]:
#Configuracion del Entrenamiento del modelo Recomendador usando la muetsra de entrenamiento
rec = ALS(
    maxIter=15,
    regParam=0.025,
    userCol='userID_cod',
    itemCol='Nombre_comerce_ID',
    ratingCol='rating',
    nonnegative=True,
    coldStartStrategy="drop",
    implicitPrefs=False,
    seed=1234
)

# maxIter: El maximo de iteraciones
# regParam: Parametro de REgularización
# nonnegative: Especificar si se utiliza una restriccion negativa en los Minimos cuadrados
# coldStartStrategy: Parametro que elimina cualquier cliente que no tenga informacion
# implicitPrefs: Si es implicito hace referencia a feedback no registrado voluntariamente por el cliente

In [ ]:
#Entrenando el modelo
rec_model = rec.fit(train)

In [ ]:
#Haciendo las predicciones en la data de test 
predicted_ratings = rec_model.transform(test)

In [ ]:
#Revisando los pronostico
predicted_ratings.printSchema()

root
 |-- userID: string (nullable = true)
 |-- userID_cod: integer (nullable = true)
 |-- placeID: integer (nullable = true)
 |-- Nombre_comerce: string (nullable = true)
 |-- Distrito: string (nullable = true)
 |-- rating_total: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- Nombre_comerce_ID: double (nullable = false)
 |-- prediction: float (nullable = false)



In [ ]:
# Revision rapida de los pronosticos
predicted_ratings.orderBy(rand()).show(10)

+------+----------+-------+--------------------+---------------+------------+------+-----------------+----------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|Nombre_comerce_ID|prediction|
+------+----------+-------+--------------------+---------------+------------+------+-----------------+----------+
| U1126|        45| 135041|           Luna Cafe|San Luis Potosi|           4|     4|              9.0|  4.023547|
| U1054|        90| 135062|Restaurante El Ci...|San Luis Potosi|           4|     4|              8.0| 4.1476994|
| U1038|        66| 132858|Hamburguesas Vall...|San Luis Potosi|           3|     3|             66.0|  3.005488|
| U1124|        46| 135032|Cafeteria y Resta...|San Luis Potosi|           3|     3|              1.0| 2.9272127|
| U1010|       129| 135047|Restaurante Casa ...|San Luis Potosi|           3|     3|             45.0| 3.5453787|
| U1004|        62| 135106|El Rinc�n de San ...|San Luis Potosi|           6|     6|    

In [ ]:
predicted_ratings.filter(predicted_ratings.userID == 'U1106').orderBy('prediction', ascending=False).show(10, False)

+------+----------+-------+-----------------------------------+---------------+------------+------+-----------------+----------+
|userID|userID_cod|placeID|Nombre_comerce                     |Distrito       |rating_total|rating|Nombre_comerce_ID|prediction|
+------+----------+-------+-----------------------------------+---------------+------------+------+-----------------+----------+
|U1106 |44        |135064 |Restaurante El Chivero S.A. de C.V.|San Luis Potosi|6           |6     |20.0             |6.1799283 |
|U1106 |44        |135064 |Restaurante El Chivero S.A. de C.V.|San Luis Potosi|6           |6     |20.0             |6.1799283 |
|U1106 |44        |135064 |Restaurante El Chivero S.A. de C.V.|San Luis Potosi|6           |6     |20.0             |6.1799283 |
|U1106 |44        |135064 |Restaurante El Chivero S.A. de C.V.|San Luis Potosi|6           |6     |20.0             |6.1799283 |
|U1106 |44        |135064 |Restaurante El Chivero S.A. de C.V.|San Luis Potosi|6           |6    

In [ ]:
_group_predict = predicted_ratings.filter(predicted_ratings.userID_cod == 44).groupBy('Nombre_comerce').agg({
     'prediction': 'mean',   'Distrito': 'last', 'rating': 'mean'
}).orderBy('avg(prediction)', ascending=False)

_group_predict.show(100, False)

+-----------------------------------+--------------------+-----------+---------------+
|Nombre_comerce                     |avg(prediction)     |avg(rating)|last(Distrito) |
+-----------------------------------+--------------------+-----------+---------------+
|Restaurante El Chivero S.A. de C.V.|6.179928302764893   |6.0        |San Luis Potosi|
|Koye Sushi                         |6.059830188751221   |6.0        |San Luis Potosi|
|La Posada del Virrey               |5.979745864868164   |6.0        |San Luis Potosi|
|Tortas Locas Hipocampo             |5.721068859100342   |6.0        |San Luis Potosi|
|Restaurante Guerra                 |4.942611217498779   |5.0        |San Luis Potosi|
|Cabana Huasteca                    |4.918063163757324   |5.0        |San Luis Potosi|
|Restaurante El Cielo Potosino      |4.849102973937988   |5.0        |San Luis Potosi|
|Preambulo Wifi Zone Cafe           |4.836749076843262   |5.0        |San Luis Potosi|
|Gorditas Doa Gloria                |3.9483

In [ ]:
_group_predict_other = predicted_ratings.filter(predicted_ratings.userID == 'U1049').groupBy('Nombre_comerce').agg({
     'prediction': 'mean',   'Distrito': 'last', 'rating': 'mean'
}).orderBy('avg(prediction)', ascending=False)

_group_predict_other.show(100, False)

+----------------------------------+---------------+-----------+---------------+
|Nombre_comerce                    |avg(prediction)|avg(rating)|last(Distrito) |
+----------------------------------+---------------+-----------+---------------+
|puesto de tacos                   |0.0            |0.0        |s.l.p.         |
|La Cantina Restaurante            |0.0            |0.0        |San Luis Potosi|
|Restaurant Oriental Express       |0.0            |0.0        |San Luis Potosi|
|Luna Cafe                         |0.0            |0.0        |San Luis Potosi|
|La Posada del Virrey              |0.0            |0.0        |San Luis Potosi|
|Cafeteria y Restaurant El Pacifico|0.0            |0.0        |San Luis Potosi|
|Tortas Locas Hipocampo            |0.0            |0.0        |San Luis Potosi|
|Restaurante Versalles             |0.0            |0.0        |San Luis Potosi|
|crudalia                          |0.0            |0.0        |0              |
+---------------------------

# 5. Evaluación del poder predictivo del modelo


In [ ]:
# Creando la columna diferencia del pronostico
predicted_ratings_witherr = _group_predict.withColumn(
    'err', abs(_group_predict['avg(prediction)'] - _group_predict['avg(rating)'])
)

predicted_ratings_witherr.show(100, False)

+-----------------------------------+--------------------+-----------+---------------+--------------------+
|Nombre_comerce                     |avg(prediction)     |avg(rating)|last(Distrito) |err                 |
+-----------------------------------+--------------------+-----------+---------------+--------------------+
|Restaurante El Chivero S.A. de C.V.|6.179928302764893   |6.0        |San Luis Potosi|0.17992830276489258 |
|Koye Sushi                         |6.059830188751221   |6.0        |San Luis Potosi|0.0598301887512207  |
|La Posada del Virrey               |5.979745864868164   |6.0        |San Luis Potosi|0.020254135131835938|
|Tortas Locas Hipocampo             |5.721068859100342   |6.0        |San Luis Potosi|0.2789311408996582  |
|Restaurante Guerra                 |4.942611217498779   |5.0        |San Luis Potosi|0.0573887825012207  |
|Cabana Huasteca                    |4.918063163757324   |5.0        |San Luis Potosi|0.08193683624267578 |
|Restaurante El Cielo Potosi

In [ ]:
redicted_ratings_witherr_other = _group_predict_other.withColumn(
    'err', abs(_group_predict_other['avg(prediction)'] - _group_predict_other['avg(rating)'])
)

redicted_ratings_witherr_other.show(100, False)

+----------------------------------+---------------+-----------+---------------+---+
|Nombre_comerce                    |avg(prediction)|avg(rating)|last(Distrito) |err|
+----------------------------------+---------------+-----------+---------------+---+
|puesto de tacos                   |0.0            |0.0        |s.l.p.         |0.0|
|La Cantina Restaurante            |0.0            |0.0        |San Luis Potosi|0.0|
|Restaurant Oriental Express       |0.0            |0.0        |San Luis Potosi|0.0|
|Luna Cafe                         |0.0            |0.0        |San Luis Potosi|0.0|
|La Posada del Virrey              |0.0            |0.0        |San Luis Potosi|0.0|
|Cafeteria y Restaurant El Pacifico|0.0            |0.0        |San Luis Potosi|0.0|
|Tortas Locas Hipocampo            |0.0            |0.0        |San Luis Potosi|0.0|
|Restaurante Versalles             |0.0            |0.0        |San Luis Potosi|0.0|
|crudalia                          |0.0            |0.0        |0

In [ ]:
predicted_ratings_witherr.agg({"err": "avg"}).collect()

[Row(avg(err)=0.15488359332084656)]

In [ ]:
# Distribución de las valoraciones
predicted_ratings.groupBy('rating').count().orderBy('rating', ascending=True).show()

+------+-----+
|rating|count|
+------+-----+
|     0| 2055|
|     1|  404|
|     2|  925|
|     3| 2485|
|     4| 1883|
|     5| 1567|
|     6| 3387|
+------+-----+



In [ ]:
# Error promedio por rating de valoracion
predicted_ratings_witherr.groupBy('avg(rating)').agg({'err':'mean'}).orderBy('avg(rating)',ascending=True).show()

+-----------+-------------------+
|avg(rating)|           avg(err)|
+-----------+-------------------+
|        0.0| 0.2774880975484848|
|        1.0|0.05713486671447754|
|        2.0|0.08525419235229492|
|        3.0|0.14157533645629883|
|        4.0| 0.1299976110458374|
|        5.0|0.11336839199066162|
|        6.0|0.13473594188690186|
+-----------+-------------------+



In [ ]:
# Error promedio por rating de valoracion
predicted_ratings_witherr.groupBy('avg(rating)').agg({'err':'std'}).orderBy('avg(rating)',ascending=True).show()

+-----------+--------------------+
|avg(rating)|         stddev(err)|
+-----------+--------------------+
|        0.0| 0.20788072708373312|
|        1.0|                 NaN|
|        2.0|                 NaN|
|        3.0|   0.077085911576849|
|        4.0| 0.11078939763185862|
|        5.0|0.051698886726005214|
|        6.0| 0.11768823338349338|
+-----------+--------------------+



In [ ]:
#Importando el evaluador de Regresion RMSE 
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
#Evluando la presición del pronostico
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [ ]:
#Evluando el error de pronostico global
rmse=evaluator.evaluate(predicted_ratings)

In [ ]:
#Imprimiento el error
print(rmse)

0.1630513549884185


# 6. Recomendandor de peliculas, ya listo!

In [ ]:
#Listado de peliculas unicas
unique_movies=indexed.select('Nombre_comerce').distinct()

In [ ]:
#Total de peliculas
unique_movies.count()

129

In [ ]:
#Asignando un alias {a} la lista unica de peliculas
a = unique_movies.alias('a')

In [ ]:
# Eligiendo un cliente
user_id = 44

# Creando una tabla que contiene una pelicula por usuario
watched_movies = indexed.filter(indexed['userId'] == user_id).select('Nombre_comerce').distinct()

#Numero de peliculas ya evaluadas 
watched_movies.count()

0

In [ ]:
# Asignando un alias  {b} ha peliculas que el usuario ya vio de la tabla
b=watched_movies.alias('b')

In [ ]:
#Cruzando ambas tablas
total_movies = a.join(b, a.title_new == b.title_new,how='left')

In [ ]:
total_movies.show(10,False)

+---------+---------+
|title_new|title_new|
+---------+---------+
|558.0    |null     |
|305.0    |null     |
|299.0    |null     |
|596.0    |null     |
|769.0    |null     |
|934.0    |null     |
|496.0    |null     |
|1051.0   |null     |
|692.0    |null     |
|810.0    |null     |
+---------+---------+
only showing top 10 rows



In [ ]:
# Seleccionando peliculas que el cliente aun no ha mirado
remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()

In [ ]:
remaining_movies.show()

+---------+
|title_new|
+---------+
|    558.0|
|    305.0|
|    299.0|
|    596.0|
|    769.0|
|    934.0|
|    496.0|
|   1051.0|
|    692.0|
|    810.0|
|    720.0|
|    782.0|
|    184.0|
|    147.0|
|    576.0|
|    170.0|
|   1369.0|
|   1587.0|
|    608.0|
|    735.0|
+---------+
only showing top 20 rows



In [ ]:
# Numero de peliculas que los usuario aun no lo ven 
remaining_movies.count()

1605

In [ ]:
# Agregando una nueva columna del usuario 
remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))

In [ ]:
remaining_movies.show(10,False)

+---------+------+
|title_new|userId|
+---------+------+
|558.0    |100   |
|305.0    |100   |
|299.0    |100   |
|596.0    |100   |
|769.0    |100   |
|934.0    |100   |
|496.0    |100   |
|1051.0   |100   |
|692.0    |100   |
|810.0    |100   |
+---------+------+
only showing top 10 rows



In [ ]:
# Haciendo la recomendacion de peliculas utilizando el modelo recomendador ALS y seleccionado solo las mejores top {n} peliculas
recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False)

In [ ]:
recommendations.show(5,False)

+---------+------+----------+
|title_new|userId|prediction|
+---------+------+----------+
|1066.0   |100   |6.513842  |
|930.0    |100   |5.687636  |
|923.0    |100   |5.469397  |
|1239.0   |100   |5.418679  |
|869.0    |100   |5.2863975 |
+---------+------+----------+
only showing top 5 rows



In [ ]:
# Consiguiendo los de las peliculas sugeridas
movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)

final_recommendations=movie_title.transform(recommendations)

In [ ]:
final_recommendations.show(10,False)

+---------+------+----------+-------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                    |
+---------+------+----------+-------------------------------------------------------------------------+
|1066.0   |100   |6.513842  |Alphaville (1965)                                                        |
|930.0    |100   |5.687636  |Ma vie en rose (My Life in Pink) (1997)                                  |
|923.0    |100   |5.469397  |Indian Summer (1996)                                                     |
|1239.0   |100   |5.418679  |Swept from the Sea (1997)                                                |
|869.0    |100   |5.2863975 |Burnt By the Sun (1994)                                                  |
|840.0    |100   |5.2617555 |Ghost in the Shell (Kokaku kidotai) (1995)                               |
|1121.0   |100   |5.0466433 |Crooklyn (1994)                    

# 7. Construyendo las funcion que sugiera una pelicula recomendada segun el modelo recomendador

In [ ]:
# Funcion para recomendar las top n peliculas para cualquier usuario
def top_movies(user_id,n):
    """
    This function returns the top 'n' movies that user has not seen yet but might like 
    
    """
    # Asignación de una lista de peliculas unicas a
    a = unique_movies.alias('a')
    
    # Creando otra tabla que contiene las peliculas que ya vio el usuario
    watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new')
    
    # Asignanción de tabla de peliculas b de peliculas ya vistas 
    b=watched_movies.alias('b')
    
    # Cruce de ambas tablas
    total_movies = a.join(b, a.title_new == b.title_new,how='left')
    
    # Seleccionando peliculas que el usuario no ha visto aun 
    remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
        
    #Agregando una nueva columan del cliente  
    remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
    
    # REalizando la recomendnado de peliculas mas adecuada utilizando el Sistema de recomendador del modelo ALS y seleccionando las mejores peliculas
    recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False).limit(n)

    # Agregando los titulos de las peliculas en texto   
    movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
    final_recommendations=movie_title.transform(recommendations)
    
    # Entregando los titutlos recomendado por usuario
    return final_recommendations.show(n,False)


In [ ]:
# REvision del clientes y las 10 peliculas sugeridas para el cliente
top_movies(123,15)

+---------+------+----------+-------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                    |
+---------+------+----------+-------------------------------------------------------------------------+
|1358.0   |123   |6.666241  |Angel Baby (1995)                                                        |
|1121.0   |123   |6.456703  |Crooklyn (1994)                                                          |
|910.0    |123   |5.9415126 |Mis<U+008E>rables, Les (1995)                                            |
|739.0    |123   |5.817608  |Night on Earth (1991)                                                    |
|1184.0   |123   |5.6485686 |Maybe, Maybe Not (Bewegte Mann, Der) (1994)                              |
|1182.0   |123   |5.599687  |Kaspar Hauser (1993)                                                     |
|495.0    |123   |5.541688  |In the Company of Men (1997)       

In [ ]:
# El cliente 450 que revisa Star Wars
DS_peliculas_rating.filter((DS_peliculas_rating['rating']>4)&(DS_peliculas_rating['userId']=='450')).show()

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|   450|L.A. Confidential...|     5|
|   450|Dr. Strangelove o...|     5|
|   450|Hunt for Red Octo...|     5|
|   450|Age of Innocence,...|     5|
|   450|     Die Hard (1988)|     5|
|   450|Right Stuff, The ...|     5|
|   450|Sleepless in Seat...|     5|
|   450|   Sting, The (1973)|     5|
|   450|  Rear Window (1954)|     5|
|   450|Dead Poets Societ...|     5|
|   450|Sound of Music, T...|     5|
|   450|E.T. the Extra-Te...|     5|
|   450|Brothers McMullen...|     5|
|   450|To Kill a Mocking...|     5|
|   450|   Old Yeller (1957)|     5|
|   450|Jerry Maguire (1996)|     5|
|   450|     Backbeat (1993)|     5|
|   450|  Taxi Driver (1976)|     5|
|   450|Young Frankenstei...|     5|
|   450|It's a Wonderful ...|     5|
+------+--------------------+------+
only showing top 20 rows



In [ ]:
# REvision del clientes y las 10 peliculas sugeridas para el cliente
top_movies(450,15)

+---------+------+----------+----------------------------------+
|title_new|userId|prediction|title                             |
+---------+------+----------+----------------------------------+
|944.0    |450   |5.4597893 |Surviving Picasso (1996)          |
|1072.0   |450   |5.440328  |Radioland Murders (1994)          |
|1358.0   |450   |5.2606144 |Angel Baby (1995)                 |
|1271.0   |450   |5.168796  |Whole Wide World, The (1996)      |
|1266.0   |450   |5.114764  |Promesse, La (1996)               |
|923.0    |450   |5.1098905 |Indian Summer (1996)              |
|1023.0   |450   |5.100616  |Gay Divorcee, The (1934)          |
|581.0    |450   |5.0989923 |Koyaanisqatsi (1983)              |
|1195.0   |450   |5.0697556 |Pather Panchali (1955)            |
|988.0    |450   |5.066791  |Ruling Class, The (1972)          |
|997.0    |450   |5.0588284 |Selena (1997)                     |
|1450.0   |450   |5.0580397 |C<U+008E>r<U+008E>monie, La (1995)|
|1488.0   |450   |5.02150